In [2]:
from utils.Dataset import Dataset

Preloading cache: 100%|██████████| 20000/20000 [05:32<00:00, 60.14it/s]


In [3]:
#Full dump
dataset = Dataset(lateralize=False, reduced=False)
dataset.preload_cache()
dataset.save_dump()

In [2]:
#Reduced dump
dataset_reduced = Dataset(lateralize=False, reduced=True, n=500)
dataset_reduced.preload_cache()
dataset_reduced.save_dump("dump_cache_reduced.npz")


Preloading cache: 100%|██████████| 440/440 [00:07<00:00, 58.07it/s]


In [10]:
import sys
is_colab = 'google.colab' in sys.modules

In [12]:
print("is_colab: ",is_colab)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2684872437.py, line 1)